In [ ]:
import torch
import torch.nn as nn
from utils.pruning import prune_model, reset_model, get_prune_percentage
from utils.training import one_epoch
import torch.nn.functional as F

from htorch import layers
from models.resnet_real import ResNet18
from models.resnet_quat import ResNet18_quat
import numpy as np
from tqdm import tqdm, trange
import matplotlib.pyplot as plt
from models.lenet_300_100 import Real, Quat

GPU = torch.device('cuda:0')
seed = 21

# class Real(nn.Module):
#     def __init__(self):
#         super().__init__()
#         self.fc1 = nn.Linear(3, 1, bias=False)

#     def forward(self, x):
#         # print(self.fc1(x))
#         return self.fc1(x).flatten()


In [ ]:
model = ResNet18_quat().to(GPU)
torch.manual_seed(seed)
model.apply(reset_model)
sd1 = model.state_dict()
# print(*sd1.keys(), sep='\n')
optimiser = torch.optim.Adam(model.parameters(), lr=0.001)
loss_fn = nn.CrossEntropyLoss()

In [ ]:
torch.manual_seed(seed)
# batch_x = torch.randn(3, 3).float().to(GPU)
# batch_y = torch.randn(3, 1).flatten().to(GPU)

batch_x = torch.randn(256, 4, 224, 224).float().to(GPU)
batch_y = torch.randint(0, 1000, (256, 1)).flatten().to(GPU)
# batch_x, batch_y

In [ ]:
# model.state_dict()

In [ ]:
losses = []
for i in trange(20):
    loss = one_epoch(model, batch_x, batch_y, optimiser, loss_fn, GPU)
    losses.append(loss.item())
plt.plot(losses)
# print((model(batch_x), batch_y))

In [ ]:
torch.manual_seed(seed)
prune_model(model, 0.67)
# model.state_dict()

In [ ]:
torch.manual_seed(seed)
model = model.apply(reset_model)
sd2 = model.state_dict()

In [ ]:
# model1 and model2
# sd1 = list(model1.state_dict())  # unpruned
# sd2 = list(model2.state_dict())  # pruned

sdk1 = [a for a in sd1.keys() if "weight" in a]
sdk2 = [a for a in sd2.keys() if "weight" in a]

for key in sdk1:
    key2 = key+"_orig"
    if sd2.get(key2) is None:
        print(f"{key2} NOT pruned")
        continue
    if torch.allclose(sd1[key], sd2[key2]):
        # print(f"{key} and {key2} are same")
        pass
    else:
        print(f"{key} and {key2} are different!")
        break